## 1. Assignment 4
Write a python code for the following
1. Read the netlist and sort the nets in the toplogical order.
2. Read the list of input vectors, evaluate the circuit and find the state of all nets in the circuit.
• Using topological sort and multiple rounds of circuit evaluations.
• Using event driven simulation.
3. Briefly discuss your results: which approach is faster/more efficient, and for what types of
inputs. Your discussion should be brief (not more than 1 page) but should give logical reasons
rather than just blanket statements.


1.1 TOPOLOGICAL SORT

In [32]:
def topological_sort(datafile):
    import numpy as np
    import networkx as nx
    nets = []
    output_nets = {}
    inputs = []
    outputs = []
    with open(datafile + ".net" ,'r') as f:
        for line in f:
            token=line.strip().split()

            if len(token) == 5:
                nets.append((token[2],token[4]))
                nets.append((token[3],token[4]))
                output_nets[token[4]] = token[1]
                inputs.append(token[2])
                inputs.append(token[3])
                inputs.append(token[4])
                outputs.append(token[4])
            if len(token) == 4:
                nets.append((token[2],token[3]))
                output_nets[token[3]] = token[1]
                inputs.append(token[2])
                inputs.append(token[3])
                outputs.append(token[3])
            
    inputs = set(inputs)
    for net in outputs:
        inputs.remove(net)
        
    inputs = list(inputs)
    inputs.sort()
    input_nets = {}
    for i in range(len(inputs)):
        input_nets[inputs[i]] = 'PI'
    gates = dict(input_nets, **output_nets)
    all_nets = inputs + outputs
    Exception_Raised = False
    try:
        # create a DAG
        g = nx.DiGraph()
        # add nodes and edges to the DAG
        g.add_edges_from(nets)
        nx.set_node_attributes(g, gates,name="gateType")
        # sort the nodes in topological order
        nl = list(nx.topological_sort(g))
    except:
        Exception_Raised = True
        return ("ERROR! Graph contains a cycle or graph changed during iteration")
#Creating a dictionary to store the current values of each Net and Initialise it to zeros
    values = {}
    for i in range(len(nl)):
        values[nl[i]] = 0
    return [nl,values,gates,g,Exception_Raised,inputs,all_nets]

Here the input to the topological sorting function is the datafile which is the file containing the netlist for the digital logic circuit,we sort the data into dictionaries and then map the input net to the output net

1.2 Defining Various Logic Gates

In [33]:
 def AND (a, b):
 
    if a == 1 and b == 1:
        return 1
    else:
        return 0
    
def NAND (a, b):
    if a == 1 and b == 1:
        return 0
    else:
        return 1
    
def OR(a, b):
    if a == 1 or b ==1:
        return 1
    else:
        return 0
    
def XOR (a, b):
    if a != b:
        return 1
    else:
        return 0
    
def NOT(a):
    return not a

def NOR(a, b):
    if(a == 0) and (b == 0):
        return 1
    elif(a == 0) and (b == 1):
        return 0
    elif(a == 1) and (b == 0):
        return 0
    elif(a == 1) and (b == 1):
        return 0
    
def XNOR(a,b):
    if(a == b):
        return 1
    else:
        return 0
    
def BUFFER(a):
    return a

`Reading from the input file

In [34]:
def read_input(datafile):
    f = open(datafile + ".inputs" ,'r')
    test_case = f.readlines()
    for i in range(len(test_case)):
        test_case[i] = test_case[i].split()
    for i in range(1, len(test_case)):
        for j in range(0,len(test_case[0])):
            test_case[i][j] = int(test_case[i][j])
    return test_case

1.3 Circuit solving for each Node

In [35]:
import numpy as np
def digitalgate(nl,values,test_case,filename,gates,g):
    output_array = np.zeros([len(test_case)-1 ,len(nl)] )
    output_array = list(output_array)
    for i in range(len(output_array)):
        output_array[i] = list(output_array[i])
    output_array.insert(0,nl)
    for j in range(1,len(test_case)):
        for i in range(len(nl)):
            if len(list(g.predecessors(nl[i]))) == 0:
                ind = test_case[0].index(nl[i])
                values[nl[i]] = test_case[j][ind]
            if len(list(g.predecessors(nl[i]))) == 1:
                temp = list(g.predecessors(nl[i]))
                if gates[nl[i]] == 'inv':
                    values[nl[i]] = NOT(values[temp[0]])
                if gates[nl[i]] == 'buf':
                    values[nl[i]] = BUFFER(values[temp[0]])
                if len(list(g.predecessors(nl[i]))) == 2:
                    temp = list(g.predecessors(nl[i]))
                if gates[nl[i]] == 'nand2':
                    values[nl[i]] = NAND(values[temp[0]],values[temp[1]])
                if gates[nl[i]] == 'nor2':
                    values[nl[i]] = NOR(values[temp[0]],values[temp[1]])
                if gates[nl[i]] == 'or2':
                    values[nl[i]] = OR(values[temp[0]],values[temp[1]])
                if gates[nl[i]] == 'and2':
                    values[nl[i]] = AND(values[temp[0]],values[temp[1]])
                if gates[nl[i]] == 'xor2':
                    values[nl[i]] = XOR(values[temp[0]],values[temp[1]])
                if gates[nl[i]] == 'xnor2':
                    values[nl[i]] = XNOR(values[temp[0]],values[temp[1]])
            n = output_array[0].index(nl[i])
            output_array[j][n] = values[nl[i]]
#Output the Truth table
    truth_table = output_array
    for i in range(len(truth_table)):
        truth_table[i] = np.array(truth_table[i])

    truth_table = np.array(truth_table)
    f = open ((filename + ".txt"),'w')
    for j in range(len(truth_table)):
        for i in range(len(truth_table[0])):
            f.write(f' {truth_table[j][i] }')
        f.write('\n')
    f.close()
    return values

Final Code for the Topological Sort solving

In [36]:
def Logic_simulation(datafile,filename):
    topo_list = topological_sort(datafile)
    nl = topo_list[0]
    values = topo_list[1]
    gates = topo_list[2]
    g = topo_list[3]
    Exception_Raised = topo_list[4]
    if Exception_Raised == False:
        test_case = read_input(datafile)
    else:
        return("ERROR! Graph contains a cycle or graph changed during iteration")
    truth_table = digitalgate(nl,values,test_case,filename,gates,g)
    return truth_table

The above code we 1st unpack the data from the topological sort function read the .input filr and put the values into a matrix -we create an output matrix to store the states of all the nets A dictionary values stores the current state of each net we check each net in the topologically sorted array If it is a primary input we just update its value Else we get the gate of which it is an output and evaluate the net using the predefined functions and the predecessors.The values in being continuously updated while the outputs is updated once the evaluation of each input vector line is complete

 The function in the above cell returns the final output. All the required functions defined above
are called inside this function

Simulation 1 for NETLIST C17

In [37]:
print('The final state of the outputs (Topological sort) is:')
print(Logic_simulation('c17','c17_output'))
%timeit Logic_simulation('c17','c17_output')

The final state of the outputs (Topological sort) is:
{'N2': 0, 'N7': 0, 'N1': 0, 'N3': 1, 'N6': 1, 'n_0': 0, 'n_1': 0, 'n_3': 0, 'n_2': 0, 'N22': 0, 'N23': 0}
842 µs ± 46.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Simulation 2 for NETLIST C17_1

In [38]:
print('The final state of the outputs (Topological sort) is:')
print(Logic_simulation('c17_1','c17_1_output'))
%timeit Logic_simulation('c17_1','c17_1_output')

The final state of the outputs (Topological sort) is:
ERROR! Graph contains a cycle or graph changed during iteration
102 µs ± 2.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Simulation 3 for NETLIST C8

In [39]:
print('The final state of the outputs (Topological sort) is:')
print(Logic_simulation('c8','c8_output'))
%timeit Logic_simulation('c8','c8_output')

The final state of the outputs (Topological sort) is:
{'q': 1, 'h': 1, 'g': 0, 'f': 1, 'bn0': 1, 'e': 1, 'an0': 1, 'd': 1, 'r': 0, 'cn0': 1, 'y': 0, 'z': 1, 'c': 1, 'b': 1, 'a': 1, 'v': 1, 'w': 0, 'x': 1, 'i': 1, 'l': 0, 'k': 1, 'm': 1, 'n': 0, 'j': 0, 'p': 1, 'u': 0, 'o': 0, 's': 1, 'n_1': False, 'n_0': 0, 'dummy_11': 0, 'dummy_32': 0, 'dummy_37': 0, 'n_20': False, 'ln0': 1, 'dummy_28': 0, 'dummy_12': 0, 'dummy_30': 0, 'n_43': False, 'dummy_39': 0, 'n_2': False, 'dummy_21': 0, 'dummy_26': 0, 'dummy_24': 0, 'n_31': False, 'dummy_34': 0, 'dummy_36': 0, 'n_5': 0, 'n_16': 0, 'n_60': 0, 'n_7': 0, 'dummy_25': 0, 'dummy_27': 0, 'dummy_29': 0, 'dummy_31': 0, 'dummy_33': 0, 'dummy_35': 0, 'dummy_38': 0, 'dummy_40': 0, 'dummy_23': 0, 'n_6': 0, 'n_25': 0, 'n_23': 0, 'n_17': 0, 'n_14': 0, 'n_12': 0, 'n_11': 0, 'n_10': 0, 'n_9': 0, 'n_8': 0, 'dummy_1': 0, 'dummy_3': 0, 'dummy_5': 0, 'dummy_7': 0, 'dummy_10': 0, 'dummy_15': 0, 'dummy_18': 0, 'dummy_20': 0, 'dummy_22': 0, 'gn0': 0, 'fn0': 0, 'hn0': 

Simulation 4 for NETLIST C432

In [40]:
print('The final state of the outputs (Topological sort) is:')
print(Logic_simulation('c432','c432_output'))
%timeit Logic_simulation('c432','c432_output')

The final state of the outputs (Topological sort) is:
{'N4': 1, 'N108': 0, 'N69': 0, 'N17': 0, 'N30': 0, 'N56': 0, 'N82': 0, 'N95': 1, 'N92': 0, 'N76': 1, 'N79': 0, 'N63': 1, 'N27': 1, 'N11': 1, 'N66': 0, 'N50': 1, 'N105': 0, 'N89': 1, 'N40': 0, 'N24': 0, 'N14': 0, 'N53': 0, 'N8': 0, 'N1': 0, 'N47': 1, 'N86': 1, 'N73': 0, 'N99': 0, 'N34': 1, 'N21': 1, 'N60': 0, 'N43': 1, 'N112': 1, 'N115': 1, 'N37': 1, 'N102': 0, 'n_2': False, 'n_4': True, 'n_8': True, 'n_3': True, 'n_1': True, 'n_9': True, 'n_5': True, 'n_0': False, 'n_6': False, 'n_7': False, 'n_20': 0, 'n_12': 0, 'n_10': 0, 'n_13': 0, 'n_14': 0, 'dummy_37': 0, 'n_15': 0, 'n_21': 0, 'n_19': 0, 'dummy_56': 0, 'n_11': True, 'dummy_58': 0, 'dummy_57': 0, 'dummy_54': 0, 'n_17': 0, 'n_16': 0, 'dummy_55': 0, 'n_18': True, 'n_22': 0, 'n_23': True, 'n_24': 0, 'n_37': 0, 'n_39': 0, 'n_34': 0, 'n_27': 0, 'n_31': 0, 'n_43': 0, 'n_26': 0, 'n_29': 0, 'N223': True, 'n_42': 0, 'n_38': 0, 'n_41': 0, 'n_40': 0, 'n_36': 0, 'n_35': 0, 'n_33': 0, 'n_28'

Simulation 5 for NETLIST PARITY 

In [41]:
print('The final state of the outputs (Topological sort) is:')
print(Logic_simulation('parity','parity_output'))
%timeit Logic_simulation('parity','parity_output')

The final state of the outputs (Topological sort) is:
{'o': 0, 'p': 1, 'k': 1, 'l': 0, 'c': 1, 'd': 1, 'h': 1, 'g': 0, 'f': 1, 'e': 1, 'n': 0, 'm': 1, 'j': 0, 'i': 1, 'b': 1, 'a': 1, 'dummy_2': 0, 'dummy_3': 0, 'dummy_4': 0, 'n_4': 0, 'n_3': 0, 'n_2': 0, 'n_1': 0, 'n_0': 0, 'dummy_1': 0, 'n_7': 0, 'n_6': 0, 'n_5': 0, 'dummy_0': 0, 'n_8': 0, 'q': 0}
1.04 ms ± 74.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
